<h1><center> CAPM Model</center></h1>
  <br>
  The Capital Asset Pricing Model (CAPM) describes the relationship        between systematic risk and expected return for assets, particularly     stocks. CAPM is widely used throughout finance for pricing risky         securities and generating expected returns for assets given the risk     of those assets and cost of capital.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.linear_model import LinearRegression
from yahoofinancials import YahooFinancials
from datetime import datetime
from dateutil.relativedelta import relativedelta
import warnings
warnings.filterwarnings('ignore')
import ipywidgets as widgets
from IPython.display import display, clear_output

<br>

### Formula for calculation of market return and stock prices:

\begin{aligned} &R_n = (C_n - C_{n-1})/ C_{n-1}  \\ &\textbf{where:} \\ &R_n = \text{return in month n} \\ &C_n = \text{price in month n} \\ \end{aligned}

<br>

### Calculation using regression:
The formula for calculating the expected return of an asset given its risk is as follows:

\begin{aligned} &ER_i = R_f + \beta_i ( ER_m - R_f ) \\ &\textbf{where:} \\ &ER_i = \text{stock return} \\ &R_f = \text{risk-free rate} \\ &R_m = \text{market return}\\ &\beta_i = \text{beta of the investment} \\ &(ER_m - R_f) = \text{market risk premium} \\ \end{aligned} 
​
​Each of these variables can be thought of using the slope-intercept framework where Re = y, B = slope, (Rm – Rf) = x, and Rf = y-intercept. Important insights to be gained from this framework are:

1. An asset is expected to generate at least the risk-free rate of return.
2. If the Beta of an individual stock or portfolio equals 1, then the return of the asset equals the average market return.
3. The Beta coefficient represents the slope of the line of best fit for each Re – Rf (y) and Rm – Rf (x) excess return pair.

In the graph above, we plotted excess stock returns over excess market returns to find the line of best fit. However, we observe that this stock has a positive intercept value after accounting for the risk-free rate. This value represents Alpha, or the additional return expected from the stock when the market return is zero.

<br>

### Calculation with covariance and variance (market):
\begin{aligned} &\beta_i = Covariance(R_e,R_m)/Variance(R_m) \\ \end{aligned}

One of the most popular uses of Beta is to estimate the cost of equity (Re) in valuation models. The CAPM estimates an asset’s Beta based on a single factor, which is the systematic risk of the market. The cost of equity derived by the CAPM reflects a reality in which most investors have diversified portfolios from which unsystematic risk has been successfully diversified away.

<br>

# Advantages of the CAPM:
The CAPM has several advantages over other methods of calculating required return, explaining why it has been popular for more than 40 years:

1. It considers only systematic risk, reflecting a reality in which most investors have diversified portfolios from which unsystematic risk has been essentially eliminated.
2. It is a theoretically-derived relationship between required return and systematic risk which has been subject to frequent empirical research and testing.
3. It is generally seen as a much better method of calculating the cost of equity than the dividend growth model (DGM) in that it explicitly considers a company’s level of systematic risk relative to the stock market as a whole.
4. It is clearly superior to the WACC in providing discount rates for use in investment appraisal

<br>

### Using Yahoo Financials module:
A powerful financial data module used for pulling fundamental and technical stock data from Yahoo Finance

We have used the NIFTY 50 index to retrieve the stock data.

<br>

<h1><center> Automatic beta Calculation for any Indian Stock (Live Demo) </center></h1>

<br>

#### Stock indexes examples: RELIANCE.NS, TCS.NS, ONGC.NS, INFY.NS, HDFCBANK.NS, etc. (from https://finance.yahoo.com/)
#### ->Minimum difference between start and end date = 1 year.
#### ->Time interval / frequency = monthly.
#### ->Stock name and data for the time period must be present in Yahoo Financial's database.

In [ ]:
# Calculation of beta using regression:

def calc_beta(Start_date, End_date, Stock, Frequency_date):
    
    yf1 = YahooFinancials('^NSEI')
    yf1_data = yf1.get_historical_price_data(start_date = Start_date, end_date = End_date, time_interval = Frequency_date)
    df1 = pd.DataFrame(yf1_data['^NSEI']['prices'])
    df1 = df1.drop('date', axis=1).rename(columns={'formatted_date': 'Date'})

    yf2 = YahooFinancials(Stock)
    yf2_data = yf2.get_historical_price_data(start_date = Start_date, end_date = End_date, time_interval = Frequency_date)
    df2 = pd.DataFrame(yf2_data[Stock]['prices'])
    df2 = df2.drop('date', axis=1).rename(columns={'formatted_date': 'Date'})
    
    df_volume = pd.DataFrame(pd.merge(df1, df2, on = 'Date')[['Date', 'volume_x', 'volume_y']])
    df_volume.dropna(inplace = True)
    df_volume.columns = ['Date', 'Market Volume', 'Stock Volume']
    
    Q1 = df_volume['Market Volume'].quantile(0.25)
    Q3 = df_volume['Market Volume'].quantile(0.75)
    IQR = Q3 - Q1 
    filter = (df_volume['Market Volume'] >= Q1 - 1.5 * IQR) & (df_volume['Market Volume'] <= Q3 + 1.5 *IQR)
    df_volume = df_volume.loc[filter] 

    Q1 = df_volume['Stock Volume'].quantile(0.25)
    Q3 = df_volume['Stock Volume'].quantile(0.75)
    IQR = Q3 - Q1 
    filter = (df_volume['Stock Volume'] >= Q1 - 1.5 * IQR) & (df_volume['Stock Volume'] <= Q3 + 1.5 *IQR)
    df_volume = df_volume.loc[filter] 
    
    df_volume['Market Volume'] = df_volume['Market Volume'].map(int)
    df_volume['Stock Volume'] = df_volume['Stock Volume'].map(int)
    df_returns = pd.DataFrame(columns = ['Market Returns', 'Stock Returns'])

    for i in range(0, len(df_volume)-1):
        df_returns.loc[i] = [100*(df_volume.iloc[i+1, 1] - df_volume.iloc[i, 1])/df_volume.iloc[i, 1], 100*(df_volume.iloc[i+1, 2] - df_volume.iloc[i, 2])/df_volume.iloc[i, 2]]

    X = df_returns['Market Returns'].values.reshape(-1, 1)
    y = df_returns['Stock Returns'].values.reshape(-1, 1)
    linear_regressor = LinearRegression()
    linear_regressor.fit(X, y)
    y_pred = linear_regressor.predict(X)
    plt.scatter(X, y)
    plt.plot(X, y_pred, color='red')
    plt.show()
    
    print("Regression Method ---->\n")
    print('Beta = ', linear_regressor.coef_[0][0], '\nIntercept = ', linear_regressor.intercept_[0])
    
    cov = np.cov(np.array([df_returns['Market Returns'], df_returns['Stock Returns']]))
    var = np.var(df_returns['Market Returns'])

    print("\nCovariance Variance Method ---->\n")
    print('\nCovariance Matrix : \n\n', cov, '\n\nVariance (Market Returns) : ', var)
    
    beta = int(cov[0][1])/var
    print('\nbeta: ',beta)

In [ ]:
def on_button_clicked(event):
    with output:
        clear_output()
        Start_date = str(start.value)
        End_date = str(end.value)
        
        try:
            datetime.strptime(Start_date, '%Y-%m-%d')
        except:
            display(widgets.HTML(value = "<h2><center>Start date in wrong format</center></h2>"))
            return
        
        s = datetime.strptime(Start_date, '%Y-%m-%d')
        
        try:
            datetime.strptime(End_date, '%Y-%m-%d')
        except:
            display(widgets.HTML(value = "<h2><center>End date in wrong format</center></h2>"))
            return
        
        e = datetime.strptime(End_date, '%Y-%m-%d')
    
        if(s>e):
            display(widgets.HTML(value = "<h2><center>Start date must be less than end date</center></h2>"))
            return
    
        if(int(relativedelta(e, s).years)<1):
            display(widgets.HTML(value = "<h2><center>Minimum difference between start and end date must be 1 year.</center></h2>"))
            return
        
        Frequency_date = "monthly"
        Stock = str(stock.value)
        
        try:
            calc_beta(Start_date, End_date, Stock, Frequency_date)
        except:
            display(widgets.HTML(value = "<h2><center>Stock name or data for the time period specified not present in Yahoo Financial's database</center></h2>"))
            return

In [ ]:
text_0 = widgets.HTML(value = "<h1></h1>", 
                     layout = widgets.Layout(
                     align_items = 'center',
                     )
                    )

start = widgets.Text(
        description = 'Start Date',
        disabled = False,
        layout = widgets.Layout(width = '25%')
    )

end = widgets.Text(
        description = 'End Date',
        disabled = False,
        layout = widgets.Layout(width = '25%')
    )

stock = widgets.Text(
        description = 'Stock',
        disabled = False,
        layout = widgets.Layout(width = '25%')
    )

calculate = widgets.Button(
            description = 'Calculate',
            layout = widgets.Layout(width = '100%')
        )


output = widgets.Output()

OutputHbox = widgets.HBox([output],
                        layout = widgets.Layout(
                        justify_content = 'center',
                        )
                    )


inputs1 = widgets.HBox([start, end, stock], 
                       layout = widgets.Layout(
                        justify_content = 'center',
                        width = '100%'
                        ))

calculate.on_click(on_button_clicked)

page = widgets.VBox([text_0, text_0, inputs1, text_0, calculate, text_0, text_0, output])
display(page)

### Beta values:
A Beta greater than 1 indicates that the security's price will be more volatile than the market and finally, a Beta less than 1 means that it will be less volatile than the market.

<b>Beta values can be roughly characterized as follows:</b>

1. b Less than 0: Negative Beta is possible but not likely. People thought gold stocks should have negative Betas but that hasn't been true.
<br>
<br>
2. b Equal to 0: Cash under your mattress, assuming no inflation!
<br>
<br>
3. b Between 0 and 1: Low-volatility investments (e.g., utility stocks).
<br>
<br>
4. b Equal to 1: Matching the index.
<br>
<br>
5. b Greater than 1: Anything more volatile than the index.
<br>
<br>
6. b Much Greater than 1: Impossible, because the stock would be expected to go to zero on any market decline.

<br>

# Result:
Beta value predicted using both methods, i.e. regression and covariance/variance approach is approximately same.